<a href="https://colab.research.google.com/github/Giffy/MongoDB_PyMongo_Tutorial/blob/master/1_2_Bicing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example 1 - Bicing stations

## Install Pymongo

In [0]:
!pip install pymongo==3.7.2   # Install PyMongo
!pip install folium geopandas    # Install folium for map visualization

## Import libraries

In [0]:
import pymongo                            # Library to access MongoDB
from pymongo import MongoClient           # Imports MongoClient 
import pandas as pd                       # Library to work with dataframes
import folium                             # Library to visualize a map

## Connect to database

In [0]:
# uri (uniform resource identifier) defines the connection parameters 
# uri = 'mongodb:// USER : PASSWORD @ SERVER_NAME : PORT / DATABASENAME')
# uri = 'mongodb:// USER : PASSWORD @ SERVER_NAME : PORT / DATABASE_NAME, CLUSTER_1_NAME : PORT , CLUSTER_2_NAME : PORT')
uri = 'localhost:27017'
# start client to connect to MongoDB server 
client = MongoClient( uri )

## Download data

In [0]:
dataset = "https://www.bicing.cat/availability_map/getJsonObject"
!wget $dataset

## Load data to database

In [0]:
!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoimport --db barcelona --collection bicing --jsonArray /content/getJsonObject

In [0]:
db = client.barcelona                     # Set the database to work on
db.list_collection_names()                # List the collections available
collection = db.bicing                    # Collection alias

## Quick overview

In [0]:
num_documents = collection.count_documents({'_id' : {'$exists' : 1}})     # Counts the documents in database
print ( 'Number of documents in database = ' + str(num_documents) )
list ( collection.find().limit(1) )                                       # Shows the first document

## Query to database:  Get active stations with at least 3 bicycles

In [0]:
# Loading database query in pandas Dataframe
filters = {'status':'OPN', 'bikes' : {'$gte' : '3' }}
fields = { '_id', 'lat' , 'lon', 'bikes', 'slots'}

query = list( collection.find( filters , fields ) )
df = pd.DataFrame ( query )

In [0]:
print ( 'Numer of active stations with at least 3 bicycles: ' + str(len (query)) )

In [0]:
# prints the first dataframe row 
df.iloc[0]

## Mark Bicing stations in map

In [0]:
center_lat = 41.394055
center_lon = 2.181299

locationmap = folium.Map(location=[ center_lat , center_lon ], zoom_start=16 ) #, width=600, height=400 )

longitud  = len( df )
for i in range ( longitud ):
    lng = float(df.iloc[i][3])
    lat = float(df.iloc[i][2])
    description = 'Bikes: ' + str(df.iloc[i][1]) + '<br> Empty slots: ' + str(df.iloc[i][4])
    folium.Marker( [ lat , lng ],
                 popup= description,
                 icon=folium.Icon(color='red')).add_to(locationmap)

locationmap